In [19]:
import os, uuid
import pandas as pd
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

### Establishing connection with Azure

In [20]:
# Retrieve the connection string for use with the application. The storage
# connection string is stored in an environment variable on the machine
# running the application called AZURE_STORAGE_CONNECTION_STRING. If the environment variable is
# created after the application is launched in a console or with Visual Studio,
# the shell or application needs to be closed and reloaded to take the
# environment variable into account.
connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

### Utilizing DefaultAzureCredential

In [3]:
account_url = "https://inappjson.blob.core.windows.net"
default_credential = DefaultAzureCredential()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient(account_url, credential=default_credential)

Sample blob download - one file into one location

In [4]:
# Download the blob to a local file
local_path = "assets/jsons/"
# Add 'AFS' before the .json extension to see both files in the data directory
download_file_path = os.path.join(local_path, str.replace('err_indata.json','.json','AFS.json')) 
container_client = blob_service_client.get_container_client(container='jsons') 
print("\nDownloading blob to \n\t" + download_file_path)

with open(file=download_file_path, mode="wb") as download_file:
 download_file.write(container_client.download_blob('test1.json').readall())


	assets/jsons/err_indataAFS.json


HttpResponseError: This request is not authorized to perform this operation using this permission.
RequestId:6ddd4100-c01e-0056-199a-1834ce000000
Time:2023-11-16T14:36:09.6040874Z
ErrorCode:AuthorizationPermissionMismatch
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AuthorizationPermissionMismatch</Code><Message>This request is not authorized to perform this operation using this permission.
RequestId:6ddd4100-c01e-0056-199a-1834ce000000
Time:2023-11-16T14:36:09.6040874Z</Message></Error>

### More advanced download:

1. Creating container_client 
2. Listing blobs inside the container `jsons`
3. Iterating over downlading from container into `local_path`

In [25]:
# Download the blob to a local file
local_path = "assets/jsons/"
try:
    os.mkdir(local_path)
except:
    print("Folder already exists. Proceeding to downlad")
# Read the container for filenames:
container_client = blob_service_client.get_container_client(container='jsons')
filenames = container_client.list_blobs()
# Add 'AFS' before the .json extension to see both files in the data directory
for blob in filenames:
    with open('{}{}'.format(local_path, blob.name), 'w') as fp: 
        pass
    download_file_path = os.path.join(local_path, str.replace(blob.name ,'.json', 'AFS.json'))
    print("\nDownloading blob to \n\t" + download_file_path)
    with open(file=download_file_path, mode="wb") as download_file:
        download_file.write(container_client.download_blob(blob.name).readall())

Folder already exists. Proceeding to downlad

	assets/jsons/err_indataAFS.json

	assets/jsons/indataAFS.json

	assets/jsons/test1AFS.json


### Deleting files in the container

In [23]:
filenames = container_client.list_blobs()

for blob in filenames:
    container_client.delete_blob(blob.name)
    print("Deleted blob:{}".format(blob.name))

Deleted blob:err_indata.json
Deleted blob:indata.json
Deleted blob:test1.json


### Uploading files into the container

In [28]:
import glob, json
uploads = glob.glob('*.json')
print(uploads)
for js in uploads:
    blob = json.load(open(js))
    blob_client = blob_service_client.get_blob_client(container='jsons', blob=js)
    blob_client.upload_blob(blob, blob_type="BlockBlob")

['err_indata.json', 'indata.json', 'test1.json']


TypeError: Unsupported data type: <class 'dict'>